In [1]:
using TOML

In [2]:
myfile = TOML.parse("""[sectionname]
key1 = "value"

[sectionname.subsectionname]
key2 = "value"
    """)

myfile["sectionname"]["subsectionname"]["key2"]

"value"

In [25]:
import Base: getindex



rec_index (generic function with 1 method)

In [63]:
module EmmysTOML

rec_index(d::Dict{String, Any}, keys::AbstractString ...) = begin
    n = length(keys)
    rec_index(d[keys[1]], keys[2:length(keys)] ...)::Dict{<:Any, <:Any}
end

rec_index(d::Dict{String, Any}, key::AbstractString) = d[key]

function parse(raw::String)
    main_dct::Dict{String, Any} = Dict{String, Any}()
    selected_section::Dict{String, Any} = main_dct
    for x in split(raw, "\n")
        if contains(x, "[")
            selected_section = Dict{String, Any}()
            section_name = x[2:length(x) - 1]
            if contains(section_name, ".")
                splts = split(section_name, ".")
                n = length(splts)
                section_name = splts[n]
                push!(rec_index(main_dct, splts[1:n - 1] ...), section_name => selected_section)
                continue
            end
            push!(main_dct, section_name => selected_section)
        else
            if length(x) < 2
                continue
            end
            key_value = split(replace(x, " " => ""), "=")
            push!(selected_section, key_value[1] => key_value[2])
        end
    end
    main_dct
end

end

Main.EmmysTOML

In [64]:
sample_toml = """
nosection = 5
[sample]
emmy = 5
clarence = 2
steve = 7
charise = 3
[sample.count]
charise = 2
clarence = 1
emily = 5
[sample.count.apples]
emmy = 2
sammy = 5
harry = 12
[other]
john = 5
emily = 8
"""

"nosection = 5\n[sample]\nemmy = 5\nclarence = 2\nsteve = 7\ncharise = 3\n[sample.count]\ncharise = 2\nclarence = 1\nemily = 5\n[sample.count.apples]\nemmy = 2\nsammy = 5\nharry = 12\n[other]\njohn = 5\nemily = 8\n"

In [65]:
samp2 = EmmysTOML.parse(sample_toml)

Dict{String, Any} with 3 entries:
  "nosection" => "5"
  "other"     => Dict{String, Any}("john"=>"5", "emily"=>"8")
  "sample"    => Dict{String, Any}("emmy"=>"5", "charise"=>"3", "count"=>Dict{S…

In [67]:
keys(samp2["sample"])

KeySet for a Dict{String, Any} with 5 entries. Keys:
  "emmy"
  "charise"
  "count"
  "steve"
  "clarence"

In [68]:
keys(samp2["sample"]["count"])

KeySet for a Dict{String, Any} with 4 entries. Keys:
  "emily"
  "charise"
  "apples"
  "clarence"

In [69]:
samp2["sample"]["count"]["apples"]

Dict{String, Any} with 3 entries:
  "emmy"  => "2"
  "sammy" => "5"
  "harry" => "12"

In [70]:
apples = samp2["sample"]["count"]["apples"]

Dict{String, Any} with 3 entries:
  "emmy"  => "2"
  "sammy" => "5"
  "harry" => "12"

In [75]:
for key in keys(apples)
    mask = [isnumeric(x) for x in apples[key]]
    f = findfirst(t -> ~(t), mask)
    if isnothing(f)
        apples[key] = parse(Int64, apples[key])
    end
end

In [76]:
apples

Dict{String, Any} with 3 entries:
  "emmy"  => 2
  "sammy" => 5
  "harry" => 12

In [77]:
"""<h1>hello world!</h1><sep></sep>"""

"<h1>hello world!</h1><sep></sep>"

In [104]:
module EmmyHTML

mutable struct Element
    tag::String
    properties::Dict{String, Any}
    text::String
end

const Elements = Vector{Element}

function parse(raw::String)
    position = findfirst("<", raw)
    elements = Elements()
    while true
        next_caret = findnext(">", raw, position[1])
        opening = raw[position[1] + 1:next_caret[1] - 1]
        open_splits = split(opening, " ")
        tag = string(open_splits[1])
        properties = Dict{String, Any}(begin
                propsplit = split(property, "=")
                propsplit[1] => propsplit[2]
        end for property in open_splits[2:length(open_splits)])
        txt_end = findnext("</$tag>", raw, next_caret[1])
        text = raw[next_caret[1] + 1:txt_end[1] - 1]
        push!(elements, Element(tag, properties, text))
        position = findnext("<", raw, txt_end[2])
        if isnothing(position)
            break
        end
    end
    elements::Elements
end

end

Main.EmmyHTML

In [105]:
sample_html = """<h1>hello friend</h1><sep></sep><p>welcome to my new page.</p>"""

"<h1>hello friend</h1><sep></sep><p>welcome to my new page.</p>"

In [106]:
elements = EmmyHTML.parse(sample_html)

3-element Vector{Main.EmmyHTML.Element}:
 Main.EmmyHTML.Element("h1", Dict{String, Any}(), "hello friend")
 Main.EmmyHTML.Element("sep", Dict{String, Any}(), "")
 Main.EmmyHTML.Element("p", Dict{String, Any}(), "welcome to my new page.")

In [155]:
module EmmyCSS

mutable struct CSSClass
    name::String
    properties::Dict{String, <:Any}
end

function parse(raw::String)
    position = findfirst("{", raw)
    classes = Vector{CSSClass}()
    while true
        name_start = findprev(" ", raw, position[1] - 2)
        if isnothing(name_start)
            name_start = 0:0
        end
        class_n = raw[name_start[1] + 1:position[1] - 1]
        propend = findnext("}", raw, position[1])
        propstring = raw[position[1] + 1:propend[1] - 1]
        propsplits = split(propstring, ";")
        properties = Dict{String, Any}(begin
                propsplit = split(prop, ":")
                string(replace(propsplit[1], " " => "", "\n" => "")) => replace(propsplit[2], " " => "", "\n" => "")
            end for prop in propsplits[1:length(propsplits) - 1])
        push!(classes, CSSClass(replace(class_n, " " => ""), properties))
        position = findnext("{", raw, propend[1])
        if isnothing(position)
            break
        end
    end
    classes
end

end

Main.EmmyCSS

In [156]:
css =  """div.main {
    background-color: red;
    border-radius: 5px;
}
div.header {
    background-color: white;
    border-weight: 12px;
    border-style: solid;
    border-color: green;
}
"""

"div.main {\n    background-color: red;\n    border-radius: 5px;\n}\ndiv.header {\n    background-color: white;\n    border-weight: 12px;\n    border-style: solid;\n    border-color: green;\n}\n"

In [157]:
EmmyCSS.parse(css)

2-element Vector{Main.EmmyCSS.CSSClass}:
 Main.EmmyCSS.CSSClass("div.main", Dict{String, Any}("background-color" => "red", "border-radius" => "5px"))
 Main.EmmyCSS.CSSClass("5px;\n}\ndiv.header", Dict{String, Any}("border-color" => "green", "background-color" => "white", "border-style" => "solid", "border-weight" => "12px"))